#  Building an Advanced RAG System 

Retrieval-Augmented Generation (RAG) system.

## What is RAG?
RAG = **Retrieval-Augmented Generation**
- **Retrieval**: Find relevant information from documents
- **Augmented**: Add that information to the AI's context
- **Generation**: AI generates an answer using that context



---
## Step 1: Import Libraries

Let's import all the tools we need:

In [1]:
from pathlib import Path           # To work with file paths easily
from openai import OpenAI           # To talk to Ollama (local AI)
from dotenv import load_dotenv      # To load environment variables
from pydantic import BaseModel, Field  # To structure our data (like creating classes)
from chromadb import PersistentClient  # Vector database to store embeddings
from tqdm import tqdm               # Progress bar to show how much work is done
from litellm import completion      # To call different AI models easily
import numpy as np                  # For math operations on arrays
from sklearn.manifold import TSNE   # To visusalize high-dimensional data in 2D/3D
import plotly.graph_objects as go   # To create interactive charts

## Step 1: Setup & Configuration

**What i did:** Import required libraries and configure the RAG system parameters.

**Key Configuration:**
- **Model:** Llama 3.1 8B via Ollama
- **Vector Database:** ChromaDB with nomic-embed-text embeddings  
- **Chunk Size:** ~300 characters with 25% overlap
- **Advanced Features:** Query classification, hybrid search, reranking, verification

In [ ]:
# 🎯 Enhanced Configuration for Better Results
MODEL="ollama/llama3.1:8b "
db_name="pre_processed3_db"
embedding_model="nomic-embed-text"
collection_name="docs"
knowledg_path=Path("../knowledge-base")
average_chunk_size=300
openai=OpenAI(base_url="http://localhost:11434/v1",api_key="ollama")

# Advanced RAG Configuration
RETRIEVAL_K_DEFAULT = 20  
RETRIEVAL_K_COUNTING = 40  
RETRIEVAL_K_SPECIFIC = 15  
CONTEXT_LIMIT = 8  
MIN_RELEVANCE_SCORE = 0.3  
USE_RERANKING = True 
ENABLE_QUERY_EXPANSION = True 
ENABLE_VERIFICATION = True  

---

## Step 2: Data Models & Structures

**What i did:** Define Pydantic models to structure our data.

**Models:**
- `Result`: Stores retrieved chunks with content and metadata
- `Chunk`: Represents AI-generated chunks (headline + summary + original text)
- `Chunks`: Collection of Chunk objects
- `RankOrder`: Structure for reranking results

---
## Step 3: Define Data Structures

We create classes to organize our data properly:

In [4]:
class Result(BaseModel):
    page_content: str
    metadata:dict

**What is `Result`?**
- A simple container that holds:
  - `page_content`: The text itself
  - `metadata`: Information about where it came from (file path, document type)

In [5]:
class Chunk(BaseModel):
    """Represents a smart chunk created by AI"""
    headline: str = Field(description="A brief heading for this chunk")
    summary: str = Field(description="A few sentences summarizing the content")
    original_text: str = Field(description="The original text from the document")

    def as_result(self, document):
        """Convert this Chunk to a Result object"""
        metadata = {"source": document["source"], "type": document["type"]}
        # Combine headline + summary + original text into one
        return Result(
            page_content=self.headline + "\n\n" + self.summary + "\n\n" + self.original_text,
            metadata=metadata
        )

class Chunks(BaseModel):
    """A list of Chunk objects"""
    chunks: list[Chunk]

**Why do we need `Chunk`?**

Instead of just splitting documents randomly, we use AI to create **smart chunks** with:
1. **Headline**: A title for quick understanding
2. **Summary**: What's in this chunk
3. **Original text**: The actual content

This helps the AI understand context better when searching!

---
## Step 4: Load Documents from Files

Now let's read all the markdown files from our knowledge base:

In [4]:
def fetch_documents():
    """Load all .md files from the knowledge-base folder"""
    documents = []

    # Go through each folder (products, employees, contracts, company)
    for folder in knowledg_path.iterdir():
        doc_type = folder.name  # Get folder name (e.g., "employees")
        
        # Find all .md files in this folder and subfolders
        for file in folder.rglob("*.md"):
            with open(file, "r", encoding="utf-8") as f:
                documents.append({
                    "type": doc_type,           # What kind of document
                    "source": file.as_posix(),  # Full file path
                    "text": f.read()            # The content
                })

    print(f"Loaded {len(documents)} documents")
    return documents

**What does this function do?**

1. Looks in the `knowledge-base` folder
2. For each subfolder (products, employees, contracts, company):
   - Finds all `.md` (markdown) files
   - Reads the content
   - Stores: type, source path, and text content
3. Returns a list of all documents

**Example output**: 76 documents loaded (employee bios, product info, contracts, etc.)

In [7]:
# Load all documents
documents = fetch_documents()

Loaded 76 documents


---
## Step 5: Split Documents into Smart Chunks Using AI

Instead of blindly splitting text, we ask AI to intelligently divide documents:

In [8]:
def make_prompt(document):
    """Create a prompt asking AI to split the document into chunks"""
    
    # Calculate how many chunks we might need
    how_many = (len(document["text"]) // average_chunk_size) + 1
    
    return f"""
You take a document and you split it into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: {document["type"]}
The document has been retrieved from: {document["source"]}

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks.
This document should probably be split into {how_many} chunks, but you can have more or less as appropriate.
There should be overlap between the chunks (about 25% overlap or about 50 words) for best retrieval results.

For each chunk, provide a headline, a summary, and the original text of the chunk.

Here is the document:

{document["text"]}

Respond with the chunks.
"""

**Why overlap chunks?**

Imagine a document about "John Smith" where his education is at the end of one chunk and his work experience at the start of the next. 

With **overlap**, both chunks contain some shared text, so:
- We won't miss important context that spans chunk boundaries
- The AI can better understand the full picture

In [9]:
def make_messages(document):
    """Format the prompt as a chat message for the AI"""
    return [
        {"role": "user", "content": make_prompt(document)},
    ]

In [ ]:
def process_document(document):
    """Use AI to split one document into smart chunks"""
    
    # 1. Create the prompt
    messages = make_messages(document)
    
    # 2. Call the AI (Llama 3.2) and ask for structured output (Chunks format)
    response = completion(model=MODEL, messages=messages, response_format=Chunks)
    
    # 3. Extract the AI's response (JSON format)
    reply = response.choices[0].message.content
    
    # 4. Parse JSON into Chunk objects
    doc_as_chunks = Chunks.model_validate_json(reply).chunks
    
    # 5. Convert each Chunk to Result format
    return [chunk.as_result(document) for chunk in doc_as_chunks]

**What happens in `process_document`?**

1. Take one document
2. Ask AI: "Please split this document into smart chunks with headlines and summaries"
3. AI returns structured JSON with chunks
4. Convert JSON to Python objects (Result)
5. Return list of chunks ready for storage

In [ ]:
def create_chunks(documents):
    """Process ALL documents and create chunks from each"""
    chunks = []
    
    # Process each document (with progress bar)
    for doc in tqdm(documents):
        chunks.extend(process_document(doc))  # Add all chunks from this doc
    
    return chunks



For 76 documents, this will:
- Call the AI 76 times
- Each call takes 5-30 seconds
- Total time: 30-60 minutes

But the result is worth it - we get intelligently chunked documents!

In [ ]:
# Create chunks from all documents (this takes a while!)
chunks = create_chunks(documents)
print(f"Created {len(chunks)} chunks from {len(documents)} documents")

  0%|          | 0/76 [00:00<?, ?it/s]

---
## Step 6: Convert Chunks to Vectors (Embeddings)

Now we convert text to numbers so computers can understand meaning:

In [12]:
def create_embeddings(chunks):
    """Convert text chunks to vectors and store in ChromaDB"""
    
    # 1. Connect to ChromaDB (vector database)
    chroma = PersistentClient(path=db_name)
    
    # 2. Delete old collection if it exists (fresh start)
    if collection_name in [c.name for c in chroma.list_collections()]:
        chroma.delete_collection(collection_name)

    # 3. Extract text from chunks
    texts = [chunk.page_content for chunk in chunks]
    
    # 4. Convert text to embeddings (vectors) using nomic-embed-text model
    emb = openai.embeddings.create(model=embedding_model, input=texts).data
    vectors = [e.embedding for e in emb]  # Each embedding is a list of 768 numbers

    # 5. Create or get collection
    collection = chroma.get_or_create_collection(collection_name)

    # 6. Prepare data for storage
    ids = [str(i) for i in range(len(chunks))]    # Unique ID for each chunk
    metas = [chunk.metadata for chunk in chunks]  # Metadata (source, type)

    # 7. Store everything in ChromaDB
    collection.add(ids=ids, embeddings=vectors, documents=texts, metadatas=metas)
    print(f"Vectorstore created with {collection.count()} documents")

**What are embeddings/vectors?**

- Text like "John works at Insurellm" becomes a list of 768 numbers: `[0.23, -0.45, 0.12, ...]`
- Similar meanings = similar numbers
- This lets us search by **meaning** not just keywords!

**Example:**
- Query: "Who is employed at the company?"
- Will match: "John works at Insurellm" (even though the words are different!)

**ChromaDB** is a vector database that:
- Stores these number lists
- Can quickly find similar vectors
- Saves to disk so we don't lose our work

In [13]:
# Create embeddings and store in ChromaDB
create_embeddings(chunks)

Vectorstore created with 1083 documents


---
## Step 7: Visualize the Vector Space

Let's see how our documents are organized in vector space:

In [14]:
# Load data from ChromaDB
chroma = PersistentClient(path=db_name)
collection = chroma.get_or_create_collection(collection_name)
result = collection.get(include=['embeddings', 'documents', 'metadatas'])

vectors = np.array(result['embeddings'])      # All embeddings (768 dimensions each)
documents = result['documents']                # The text
metadatas = result['metadatas']                # Metadata (source, type)
doc_types = [metadata['type'] for metadata in metadatas]  # Extract types

# Assign colors based on document type
color_map = {'products': 'blue', 'employees': 'green', 'contracts': 'red', 'company': 'orange'}
colors = [color_map[t] for t in doc_types]

**TSNE: Making 768 dimensions visible**

Our vectors have 768 numbers (dimensions). We can't visualize that!

**TSNE** (t-Distributed Stochastic Neighbor Embedding) squashes 768 dimensions down to 2D or 3D while preserving relationships:
- Documents about similar topics stay close together
- Different topics separate

In [15]:
# Reduce to 2D for visualization
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create interactive 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Vector Store Visualization',
    xaxis_title='Dimension 1',
    yaxis_title='Dimension 2',
    width=800, height=600
)

fig.show()

**What you'll see:**
- Blue dots (products) clustered together
- Green dots (employees) in their own area
- Red dots (contracts) grouped
- Orange dots (company info) separate

This shows that our embeddings successfully capture document types and meanings!

---
## Step 8: Build the RAG System

Now we create the retrieval and answering system!

### 8.1: Basic Retrieval (without reranking)

In [28]:
RETRIEVAL_K = 10  # How many chunks to retrieve

def fetch_context_unranked(question):
    """Retrieve relevant chunks for a question (no reranking yet)"""
    
    # 1. Convert question to vector
    query = openai.embeddings.create(model=embedding_model, input=[question]).data[0].embedding
    
    # 2. Search for similar vectors in ChromaDB
    results = collection.query(query_embeddings=[query], n_results=RETRIEVAL_K)
    
    # 3. Convert results to Result objects
    chunks = []
    for result in zip(results["documents"][0], results["metadatas"][0]):
        chunks.append(Result(page_content=result[0], metadata=result[1]))
    
    return chunks

**How retrieval works:**

1. User asks: "Who won the IIOTY award?"
2. Convert question to vector (768 numbers)
3. Compare with all document vectors using **cosine similarity**
4. Return top 10 most similar chunks

**Cosine similarity**: Measures how "close" two vectors are (0 = completely different, 1 = identical)

### 8.2: Reranking - Making Results Better

In [29]:
class RankOrder(BaseModel):
    """Structure for the reranked order"""
    order: list[int] = Field(
        description="The order of relevance of chunks, from most to least relevant, by chunk id number"
    )

In [ ]:
# 🎯 ENHANCEMENT 1: Query Classification
def classify_question(question):
    """Classify question type to determine retrieval strategy"""
    question_lower = question.lower()
    
    # Counting/aggregation questions
    if any(word in question_lower for word in ["how many", "count", "number of", "total"]):
        return "counting", RETRIEVAL_K_COUNTING
    
    # Specific fact questions (names, dates, specific info)
    elif any(word in question_lower for word in ["who is", "what is", "when", "date of birth", "email", "phone"]):
        return "specific", RETRIEVAL_K_SPECIFIC
    
    # Comparative/analytical questions
    elif any(word in question_lower for word in ["compare", "difference", "vs", "versus", "better"]):
        return "comparative", RETRIEVAL_K_DEFAULT
    
    # General/broad questions
    else:
        return "general", RETRIEVAL_K_DEFAULT

In [ ]:
# 🎯 ENHANCEMENT 2: Query Expansion for Better Retrieval
def expand_query(question):
    """Generate multiple query variations to improve retrieval coverage"""
    if not ENABLE_QUERY_EXPANSION:
        return [question]
    
    variations = [question]
    question_lower = question.lower()
    
    # Add keyword-based variations
    if "employee" in question_lower:
        variations.append(question + " staff member team")
    
    if "product" in question_lower:
        variations.append(question + " insurance offering service")
    
    if "contract" in question_lower:
        variations.append(question + " agreement partnership client")
    
    if "remote" in question_lower:
        variations.append(question.replace("remote", "work from home telecommute"))
    
    return variations

**Why rerank?**

Vector similarity (step 8.1) is good but not perfect. Sometimes:
- Chunk #7 is more relevant than chunk #2
- But vector similarity ranked #2 higher

**Reranking** uses AI to read all chunks and the question, then reorder them by true relevance.

**Example:**
- Question: "Who went to Manchester University?"
- Before reranking: Answer is in position #15
- After reranking: Answer moves to position #1!

In [ ]:
# 🎯 ENHANCEMENT 3: Hybrid Search with Metadata Filtering
def fetch_context_hybrid(question, k=RETRIEVAL_K_DEFAULT):
    """Enhanced retrieval with metadata filtering and query expansion"""
    
    # 1. Detect document type from question
    doc_type_filter = None
    question_lower = question.lower()
    
    if "employee" in question_lower or "staff" in question_lower or "team member" in question_lower:
        doc_type_filter = "employees"
    elif "product" in question_lower or "insurance" in question_lower or "offering" in question_lower:
        doc_type_filter = "products"
    elif "contract" in question_lower or "client" in question_lower or "agreement" in question_lower:
        doc_type_filter = "contracts"
    elif "company" in question_lower or "about" in question_lower or "culture" in question_lower:
        doc_type_filter = "company"
    
    # 2. Expand query for better coverage
    query_variations = expand_query(question)
    all_chunks = []
    seen_content = set()  # Track duplicates
    
    # 3. Search with each query variation
    for query_text in query_variations[:2]:  # Use top 2 variations
        query_emb = openai.embeddings.create(model=embedding_model, input=[query_text]).data[0].embedding
        
        # Search with metadata filter if applicable
        if doc_type_filter:
            results = collection.query(
                query_embeddings=[query_emb], 
                n_results=k,
                where={"type": doc_type_filter}
            )
        else:
            results = collection.query(
                query_embeddings=[query_emb], 
                n_results=k
            )
        
        # 4. Add results avoiding duplicates
        for doc, metadata, distance in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
            content_hash = hash(doc[:100])  # Use first 100 chars as hash
            if content_hash not in seen_content:
                seen_content.add(content_hash)
                # Calculate relevance score (1 - normalized distance)
                relevance = 1 / (1 + distance)  # Convert distance to similarity
                all_chunks.append({
                    "chunk": Result(page_content=doc, metadata=metadata),
                    "relevance": relevance
                })
    
    # 5. Sort by relevance and filter low-scoring chunks
    all_chunks.sort(key=lambda x: x["relevance"], reverse=True)
    filtered_chunks = [item["chunk"] for item in all_chunks if item["relevance"] >= MIN_RELEVANCE_SCORE]
    
    # 6. Return top K unique chunks
    return filtered_chunks[:k]

In [ ]:
# 🎯 ENHANCEMENT 4: Context Relevance Scoring (Simple keyword-based)
def score_chunk_relevance(chunk, question):
    """Score chunk relevance using keyword matching"""
    question_words = set(question.lower().split())
    chunk_words = set(chunk.page_content.lower().split())
    
    # Calculate overlap
    overlap = len(question_words & chunk_words)
    
    # Boost score if source type matches question keywords
    boost = 0
    if "employee" in question.lower() and chunk.metadata.get("type") == "employees":
        boost = 5
    elif "product" in question.lower() and chunk.metadata.get("type") == "products":
        boost = 5
    elif "contract" in question.lower() and chunk.metadata.get("type") == "contracts":
        boost = 5
    
    return overlap + boost

In [ ]:
def rerank(question, chunks):
    """Use AI to reorder chunks by relevance to the question"""
    
    # If reranking disabled or too few chunks, use keyword scoring
    if not USE_RERANKING or len(chunks) <= 3:
        # Simple keyword-based reranking
        scored_chunks = [(chunk, score_chunk_relevance(chunk, question)) for chunk in chunks]
        scored_chunks.sort(key=lambda x: x[1], reverse=True)
        return [chunk for chunk, _ in scored_chunks]
    
    system_prompt = """
You are a document re-ranker.
You are provided with a question and a list of relevant chunks of text.
You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.
Reply with ONLY a JSON object with an "order" key containing the list of chunk IDs.
Example: {"order": [3, 1, 5, 2, 4]}
"""
    
    # Build the prompt with all chunks (limit to top 10 for reranking speed)
    chunks_to_rerank = chunks[:10]
    user_prompt = f"Question: {question}\n\nChunks:\n\n"
    for index, chunk in enumerate(chunks_to_rerank):
        user_prompt += f"ID {index + 1}: {chunk.page_content[:300]}...\n\n"  # First 300 chars
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    # Try AI reranking with fallback
    try:
        response = completion(
            model=MODEL,
            messages=messages,
            timeout=120  # Reduced timeout
        )
        reply = response.choices[0].message.content
        
        # Parse JSON from response
        import re
        import json
        json_match = re.search(r'\{[\s\S]*\}', reply)
        if json_match:
            json_str = json_match.group(0)
            parsed_data = json.loads(json_str)
            order = RankOrder(**parsed_data).order
            
            # Return reranked chunks + remaining chunks
            reranked = [chunks_to_rerank[i - 1] for i in order if 0 < i <= len(chunks_to_rerank)]
            remaining = chunks[len(chunks_to_rerank):]
            return reranked + remaining
        else:
            raise ValueError("No JSON found in response")
            
    except Exception as e:
        print(f"⚠️  Reranking failed ({e}), using keyword scoring...")
        # Fallback to keyword scoring
        scored_chunks = [(chunk, score_chunk_relevance(chunk, question)) for chunk in chunks]
        scored_chunks.sort(key=lambda x: x[1], reverse=True)
        return [chunk for chunk, _ in scored_chunks]

In [31]:
def fetch_context(question):
    """Retrieve AND rerank chunks"""
    chunks = fetch_context_unranked(question)  # Get top 10
    return rerank(question, chunks)             # Reorder them

### 8.3: Query Rewriting - Understanding What Users Really Mean

In [32]:
def rewrite_query(question, history=[]):
    """Rewrite the user's question to be more specific for searching"""
    
    message = f"""
You are in a conversation with a user, answering questions about the company Insurellm.
You are about to look up information in a Knowledge Base.

This is the conversation history:
{history}

And this is the user's current question:
{question}

Respond with a single, refined question that you will use to search the Knowledge Base.
It should be a VERY short specific question most likely to surface content.
Focus on the question details. Don't mention the company name unless it's a general question.
IMPORTANT: Respond ONLY with the knowledge base query, nothing else.
"""
    
    response = completion(
        model=MODEL,
        messages=[{"role": "system", "content": message}],
        timeout=120  # 2 minutes for query rewriting
    )
    return response.choices[0].message.content

**Why rewrite queries?**

Users often ask questions in conversational ways that aren't optimal for search.

**Examples:**
- User: "Can you tell me who won that insurance award thing?"
  - Rewritten: "IIOTY award winner"

- User: "What about their education background?"
  - Rewritten: "Manchester University education" (uses conversation context)

This makes searches more accurate!

### 8.4: Generate the Final Answer

In [ ]:
SYSTEM_PROMPT = """
You are a knowledgeable, precise assistant representing Insurellm, an insurance company.
You are answering questions using information from the company's knowledge base.

IMPORTANT INSTRUCTIONS:
1. Base your answer ONLY on the provided context extracts below
2. If the context doesn't contain enough information, say "I don't have enough information to answer that accurately"
3. For counting questions, count carefully from the provided documents - don't make assumptions
4. Always cite which document type you're referencing (employee records, product docs, contracts, etc.)
5. Be specific and accurate - don't generalize or guess
6. If multiple interpretations exist, state the ambiguity clearly

CONTEXT FROM KNOWLEDGE BASE:
{context}

---
Now answer the user's question accurately and completely based ONLY on the context above.
If you need to count items, count them carefully from the provided information.
"""

In [ ]:
# Answer Verification
def verify_answer(question, answer, chunks):
    """Verify if answer is grounded in the retrieved context"""
    if not ENABLE_VERIFICATION:
        return answer
    
    verification_prompt = f"""
You are a quality checker. Review if the answer is properly supported by the provided context.

QUESTION: {question}

ANSWER: {answer}

CONTEXT:
{chr(10).join([f"- {chunk.page_content[:200]}..." for chunk in chunks[:5]])}

Respond with:
1. "VERIFIED" if the answer is well-supported by the context
2. "UNSUPPORTED" if the answer makes claims not in the context
3. "PARTIALLY" if some parts are supported but others aren't

Then briefly explain your assessment in 1 sentence.
Format: [STATUS] Explanation
"""
    
    try:
        response = completion(
            model=MODEL,
            messages=[{"role": "user", "content": verification_prompt}],
            timeout=60
        )
        verification = response.choices[0].message.content
        
        # If unsupported, add warning
        if "UNSUPPORTED" in verification or "PARTIALLY" in verification:
            answer += f"\n\n⚠️ Verification Note: {verification}"
    except:
        pass  # Skip verification if it fails
    
    return answer

In [34]:
def make_rag_messages(question, history, chunks):
    """Prepare messages for the AI with context from retrieved chunks"""
    
    # Combine all chunks into context
    context = "\n\n".join(
        f"Extract from {chunk.metadata['source']}:\n{chunk.page_content}" 
        for chunk in chunks
    )
    
    # Insert context into system prompt
    system_prompt = SYSTEM_PROMPT.format(context=context)
    
    # Build message list: system + history + current question
    return [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]

**What's happening here?**

We're giving the AI:
1. **System prompt**: "You are an assistant for Insurellm. Here's relevant context from documents..."
2. **Context**: All the retrieved chunks
3. **History**: Previous conversation (if any)
4. **Question**: The user's current question

This way, the AI has all the information it needs to give an accurate answer!

In [ ]:
def answer_question_enhanced(question: str, history: list[dict] = [], verbose=True):
    """
    Enhanced RAG pipeline with all improvements:
    1. Query classification
    2. Hybrid search with metadata filtering
    3. Smart reranking
    4. Context limiting
    5. Answer verification
    """
    
    if verbose:
        print(f"\n{'='*60}")
        print(f"🔍 Processing: {question}")
        print(f"{'='*60}\n")
    
    # Step 1: Classify question and determine retrieval strategy
    question_type, k_value = classify_question(question)
    if verbose:
        print(f"📊 Question Type: {question_type.upper()}")
        print(f"📥 Retrieving {k_value} chunks\n")
    
    # Step 2: Rewrite query for better search
    query = rewrite_query(question, history)
    if verbose:
        print(f"🔄 Rewritten query: '{query}'\n")
    
    # Step 3: Hybrid retrieval with metadata filtering
    chunks = fetch_context_hybrid(query, k=k_value)
    if verbose:
        print(f"✅ Retrieved {len(chunks)} unique chunks\n")
    
    # Step 4: Rerank chunks (with fallback to keyword scoring)
    if USE_RERANKING and len(chunks) > 1:
        if verbose:
            print(f"🎯 Reranking chunks...")
        chunks = rerank(query, chunks)
    
    # Step 5: Limit context to most relevant chunks
    chunks = chunks[:CONTEXT_LIMIT]
    if verbose:
        print(f"📝 Using top {len(chunks)} chunks for answer generation\n")
        print(f"📚 Source types: {set(c.metadata.get('type', 'unknown') for c in chunks)}\n")
    
    # Step 6: Build prompt with context
    messages = make_rag_messages(question, history, chunks)
    
    # Step 7: Generate answer
    if verbose:
        print(f"💭 Generating answer...\n")
    
    response = completion(
        model=MODEL,
        messages=messages,
        temperature=0.1,  # Low temperature for factual answers
        timeout=300
    )
    
    answer = response.choices[0].message.content
    
    # Step 8: Verify answer
    if ENABLE_VERIFICATION:
        if verbose:
            print(f"✓ Verifying answer...\n")
        answer = verify_answer(question, answer, chunks)
    
    if verbose:
        print(f"{'='*60}\n")
    
    return answer, chunks

---

## Step 10: Testing & Debugging

**What we do:** Test the RAG system and fix issues with accuracy.

**Tests Performed:**
- Product counting questions
- Employee location queries  
- Remote worker counting (revealed accuracy issues)

**Issues Found & Fixed:**
- Initial retrieval was too limited (K=10)
- Missing employee documents in results
- Ambiguous company overview causing wrong inferences

**Solutions Implemented:**
1. Increased RETRIEVAL_K to 25+ for counting questions
2. Added verification function with metadata filtering
3. Used `where={"type": "employees"}` for targeted queries
4. Created `answer_with_verification()` for fact-checking

In [ ]:
# Example
answer, chunks = answer_question_enhanced("how many product offers does insurellm")
print("\nAnswer:")
print(answer)
print("\nTop chunk used:")
print(chunks[0].page_content[:200], "...")

Rewritten query: How many product offers does Insurellm offer?


c:\Users\Ahmed Pasha\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='How many... reasoning_content=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...reasoning_content=None)), input_type=Choices])

c:\Users\Ahmed Pasha\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='{"order"... reasoning_content=None), input_type=Message])
  Pydantic


Answer:
According to our knowledge, Insurellm offers 8 insurance software products across multiple insurance lines. These products are: 

1. Markellm (marketplace connecting consumers with insurance providers)
2. Claimllm (AI-powered claims processing platform across all insurance lines)
3. Rellm (enterprise platform for the reinsurance sector)
4. Carllm
5. Homellm
6. Lifellm (life insurance)
7. Healthllm (health insurance)
8. Bizllm (commercial insurance)

These 8 products cater to a wide range of clients, from regional insurers to global reinsurance partners, demonstrating Insurellm's ability to compete across the entire insurance value chain.

Top chunk used:
Products

Insurellm offers 8 insurance software products across multiple insurance lines.

## Products

Insurellm offers 8 insurance software products across multiple insurance lines:
 ...


c:\Users\Ahmed Pasha\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="Accordin... reasoning_content=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...reasoning_content=None)), input_type=Choices])

